# Description du Modèle de Classification d'Images

Dans ce notebook, nous allons détailler un modèle de réseau de neurones convolutionnel (CNN) conçu pour la classification d'images. Le modèle est conçu pour distinguer deux classes : par exemple, "Sain" et "Pneumonie". Nous allons décrire l'architecture du modèle, les hyperparamètres utilisés, et les résultats d'évaluation.

## Objectifs
- Comprendre la structure du modèle CNN
- Examiner les hyperparamètres choisis
- Analyser les résultats obtenus après l'entraînement


## Architecture du Modèle

Le modèle utilisé dans ce projet est un réseau de neurones convolutionnel (CNN) construit avec `tensorflow.keras`. Il comporte plusieurs couches de convolution, de pooling et de dropout, suivies de couches entièrement connectées.

Voici la structure détaillée du modèle :


In [ ]:
from tensorflow.keras import layers, models


def model_ia(train_dataset, val_dataset, test_dataset):
    model = models.Sequential([
        layers.InputLayer(input_shape=(150, 150, 1)),

        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        layers.Flatten(),

        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),

        layers.Dense(2, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model


### Description des Couches

- **Input Layer** : Accepte des images de taille 150x150 pixels avec 1 canaux car seulement des couleurs de gris.
- **Convolutional Layers** : Trois couches de convolution avec des tailles de filtre de 3x3. Le nombre de filtres augmente à chaque couche (32, 64, 128) pour capturer des caractéristiques de plus en plus complexes.
- **Pooling Layers** : Chaque couche de convolution est suivie d'une couche de pooling (2x2) pour réduire la dimensionnalité et extraire les caractéristiques dominantes.
- **Dropout Layers** : Les couches Dropout (0.25 et 0.5) sont ajoutées pour éviter le surapprentissage en désactivant aléatoirement une fraction des neurones.
- **Dense Layers** : Une couche dense de 128 neurones avec activation ReLU, suivie d'une couche de sortie avec 2 neurones et activation Softmax pour la classification binaire.


## Hyperparamètres Utilisés

Les hyperparamètres suivants ont été définis pour contrôler le comportement du modèle :

- **`input_shape`** : `(150, 150, 1)` - La taille des images d'entrée.
- **`filters`** : `32`, `64`, `128` - Nombre de filtres dans les couches de convolution.
- **`kernel_size`** : `(3, 3)` - Taille des filtres de convolution.
- **`pool_size`** : `(2, 2)` - Taille des fenêtres de pooling.
- **`dropout_rate`** : `0.25` pour les couches après chaque convolution, `0.5` avant la dernière couche dense.
- **`units`** : `128` - Nombre de neurones dans la couche dense avant la sortie.
- **`activation`** : `relu` pour les couches intermédiaires, `softmax` pour la couche de sortie.
- **`optimizer`** : `adam` - Algorithme d'optimisation utilisé pour l'entraînement.
- **`loss`** : `categorical_crossentropy` - Fonction de perte pour la classification multiclasse.
- **`epochs`** : `[10, 50]` - Nombre de fois que l'ensemble des données d'entraînement est parcouru pendant l'entraînement. Ici nous avons tester avec 2 valeurs différentes.


## Entraînement du Modèle

Le modèle est entraîné sur les données d'entraînement (`train_dataset`) et validé sur les données de validation (`val_dataset`). Le processus d'entraînement se déroule sur 10 et 50 époques.


In [ ]:
import pandas as pd

from src.data_handling import generate_labeled_data, create_dataset
from src.model import model_ia
from src.visualization import history_plot


def main():
    # Labeliser les données
    data_dir = 'src/data/chest_Xray'

    df = generate_labeled_data(data_dir)

    output_csv_path = 'src/labeled/labeled_dataset.csv'
    df.to_csv(output_csv_path, index=False)

    # Préparer les données
    df = pd.read_csv('src/labeled/labeled_dataset.csv')

    label_map = {'Healthy': 0, 'Pneumonia': 1}

    train_df = df[df['type'] == 'train']
    val_df = df[df['type'] == 'val']
    test_df = df[df['type'] == 'test']

    train_dataset = create_dataset(train_df, label_map)
    val_dataset = create_dataset(val_df, label_map)
    test_dataset = create_dataset(test_df, label_map)

    # Entraîner le modèle
    model, history, val_loss, val_acc = model_ia(train_dataset, val_dataset, test_dataset)

    # Visualiser les résultats
    history_plot(history)


if __name__ == "__main__":
    main()

In [ ]:



def history_plot(history):
    # Summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    # Summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

## Graphique Importé

Voici les deux graphiques importé depuis une capture d'écran.

![Graphique avec 10 époques](assets/img/10epochs.png)
![Graphique avec 50 époques](assets/img/50epochs.png)





## Conclusion

Nous avons construit et entraîné un modèle CNN pour la classification d'images en deux classes. Les résultats de validation montrent que le modèle surapprend pour le moment.

Pour des améliorations futures, nous pourrions envisager :
- D'augmenter la taille des données d'entraînement pour améliorer la généralisation.
- D'ajuster davantage les hyperparamètres, notamment le nombre de filtres et les taux de dropout.
- D'utiliser des techniques d'augmentation de données pour augmenter la diversité des données d'entraînement.
